In [1]:
k = 30
l = 5
numBatches = 128
numStocksInSubset = 11
numEpisodes = 1
tranCostRate = 0.0025

In [2]:
import pandas as pd
import numpy as np
import random

import torch
import torch.optim as optim

from transformer import RATransformer

In [3]:
df = pd.read_csv("whole_selected.csv")
df

,Ticker,Date,Open,High,Low,Close,Adj Close,Volume
0,0001.HK,2010-01-04,9.240858,9.286962,9.199347,9.222406,5.898139,5.826086e+05
1,0001.HK,2010-01-05,9.342293,9.406862,9.277736,9.351518,5.980711,7.809818e+05
2,0001.HK,2010-01-06,9.471419,9.471419,9.323854,9.360745,5.986612,7.584029e+05
3,0001.HK,2010-01-07,9.360745,9.434527,9.296189,9.388410,6.004304,8.268949e+05
4,0001.HK,2010-01-08,9.314628,9.443741,9.314628,9.416076,6.021997,1.041317e+06
...,...,...,...,...,...,...,...,...
1140567,ZBRA,2020-10-16,289.959991,295.369995,289.239990,293.000000,293.000000,2.188000e+05
1140568,ZBRA,2020-10-19,292.200012,299.359985,291.279999,291.570007,291.570007,3.476000e+05
1140569,ZBRA,2020-10-20,293.239990,300.000000,292.649994,294.829987,294.829987,2.111000e+05
1140570,ZBRA,2020-10-21,295.290009,300.329987,291.190002,291.859985,291.859985,2.331000e+05


In [4]:
df = df[["Ticker", "Date", "Open", "High", "Low", "Close"]]

In [5]:
tickers = df["Ticker"].unique()
numTickers = len(tickers)
print("Number of tickers: " + str(numTickers))
tickersDict = {}
for index, ticker in enumerate(tickers):
    tickersDict[ticker] = index

df["Ticker"] = df["Ticker"].apply(lambda ticker: tickersDict[ticker])
df

Number of tickers: 422


,Ticker,Date,Open,High,Low,Close
0,0,2010-01-04,9.240858,9.286962,9.199347,9.222406
1,0,2010-01-05,9.342293,9.406862,9.277736,9.351518
2,0,2010-01-06,9.471419,9.471419,9.323854,9.360745
3,0,2010-01-07,9.360745,9.434527,9.296189,9.388410
4,0,2010-01-08,9.314628,9.443741,9.314628,9.416076
...,...,...,...,...,...,...
1140567,421,2020-10-16,289.959991,295.369995,289.239990,293.000000
1140568,421,2020-10-19,292.200012,299.359985,291.279999,291.570007
1140569,421,2020-10-20,293.239990,300.000000,292.649994,294.829987
1140570,421,2020-10-21,295.290009,300.329987,291.190002,291.859985


In [6]:
datesValueCounts = df["Date"].value_counts()
validDates = datesValueCounts.loc[datesValueCounts == max(datesValueCounts)].index
validDates = list(validDates.sort_values())
print("Number of valid dates: " + str(len(validDates)))

Number of valid dates: 1843


In [7]:
print(validDates[:100])
validDates = validDates[5:]

['2010-12-22', '2011-01-11', '2011-01-12', '2011-06-29', '2011-06-30', '2012-06-18', '2012-06-19', '2012-06-20', '2012-06-21', '2012-06-22', '2012-06-25', '2012-06-26', '2012-06-27', '2012-06-28', '2012-06-29', '2012-07-03', '2012-07-05', '2012-07-06', '2012-07-09', '2012-07-10', '2012-07-11', '2012-07-12', '2012-07-13', '2012-07-17', '2012-07-18', '2012-07-19', '2012-07-20', '2012-07-23', '2012-07-24', '2012-07-25', '2012-07-26', '2012-07-27', '2012-07-30', '2012-07-31', '2012-08-01', '2012-08-02', '2012-08-03', '2012-08-07', '2012-08-08', '2012-08-09', '2012-08-10', '2012-08-13', '2012-08-14', '2012-08-15', '2012-08-16', '2012-08-17', '2012-08-20', '2012-08-21', '2012-08-22', '2012-08-23', '2012-08-24', '2012-08-28', '2012-08-29', '2012-08-30', '2012-08-31', '2012-09-04', '2012-09-05', '2012-09-06', '2012-09-07', '2012-09-10', '2012-09-11', '2012-09-12', '2012-09-13', '2012-09-14', '2012-09-18', '2012-09-19', '2012-09-20', '2012-09-21', '2012-09-24', '2012-09-25', '2012-09-26', '2012

In [8]:
df = df[df["Date"].isin(validDates)]

In [9]:
dates = df["Date"].unique()
numDates = len(dates)
print("Number of valid dates: " + str(numDates))
datesDict = {}
for index, date in enumerate(dates):
    datesDict[date] = index

df["Date"] = df["Date"].apply(lambda date: datesDict[date])
df

Number of valid dates: 1838


,Ticker,Date,Open,High,Low,Close
603,0,0,8.646004,8.682895,8.553782,8.563008
604,0,1,8.581447,8.618338,8.470786,8.539949
605,0,2,8.586067,8.613732,8.521510,8.572234
606,0,3,8.567615,8.599899,8.503059,8.530723
607,0,4,8.530723,8.530723,8.378551,8.424669
...,...,...,...,...,...,...
1140529,421,1833,279.369995,284.549988,274.359985,275.390015
1140530,421,1834,276.220001,281.190002,274.450012,277.859985
1140531,421,1835,278.209991,286.170013,278.010010,284.739990
1140532,421,1836,287.230011,287.709991,275.529999,277.260010


In [10]:
df = df.sort_values(by=["Ticker", "Date"])
df

,Ticker,Date,Open,High,Low,Close
603,0,0,8.646004,8.682895,8.553782,8.563008
604,0,1,8.581447,8.618338,8.470786,8.539949
605,0,2,8.586067,8.613732,8.521510,8.572234
606,0,3,8.567615,8.599899,8.503059,8.530723
607,0,4,8.530723,8.530723,8.378551,8.424669
...,...,...,...,...,...,...
1140529,421,1833,279.369995,284.549988,274.359985,275.390015
1140530,421,1834,276.220001,281.190002,274.450012,277.859985
1140531,421,1835,278.209991,286.170013,278.010010,284.739990
1140532,421,1836,287.230011,287.709991,275.529999,277.260010


In [11]:
entries = df[["Open", "High", "Low", "Close"]].to_numpy()
entryArrays = entries.reshape((numTickers, numDates, 4)) # shape: (numStocks: m, numDates: T, numFeatures)
entryArrays

array([[[  8.64600362,   8.68289541,   8.55378205,   8.56300844],
        [  8.58144744,   8.61833823,   8.47078587,   8.53994935],
        [  8.58606654,   8.61373193,   8.52151035,   8.57223384],
        ...,
        [  6.6066222 ,   6.62597725,   6.54855609,   6.58726667],
        [  6.54210441,   6.59371835,   6.51629719,   6.52274888],
        [  6.52274888,   6.6388811 ,   6.51629719,   6.53565273]],

       [[  8.30989207,   8.3292476 ,   8.25827763,   8.30343989],
        [  8.37440986,   8.38731322,   8.32279543,   8.37440986],
        [  8.42602429,   8.42602429,   8.35505432,   8.38731322],
        ...,
        [  9.78734915,   9.81960805,   9.71638017,   9.7421869 ],
        [  9.7421869 ,   9.79380133,   9.69702464,   9.709928  ],
        [  9.78734915,   9.82606022,   9.709928  ,   9.76154243]],

       [[  1.10050457,   1.10712534,   1.09255848,   1.10182843],
        [  1.10580148,   1.10580148,   1.08593636,   1.08726157],
        [  1.09123462,   1.10050457,   1.08858

In [12]:
entryArraysTransposed = entryArrays.T # shape: (numFeatures, numDates: T, numStocks: m)
entryArraysClosingPrices = entryArraysTransposed[3] # shape: (numDates: T, numStocks: m)
inflations = np.array([entryArraysClosingPrices[i + 1] / entryArraysClosingPrices[i] for i in range(len(entryArraysClosingPrices) - 1)]) # shape: (numDates-1: T-1, numStocks: m)
inflations # percentage change from period i to (i+1)

array([[0.99730713, 1.00854706, 0.98677938, ..., 1.01297014, 1.00119406,
        1.00582639],
       [1.00378041, 1.00154081, 1.00365418, ..., 1.        , 0.97555159,
        1.0125    ],
       [0.99515752, 0.99461541, 0.99514471, ..., 0.97567225, 0.95599021,
        0.9728998 ],
       ...,
       [0.9893411 , 0.99276788, 0.98579042, ..., 0.99365421, 1.03452685,
        1.02476069],
       [0.99020568, 0.99668874, 0.99459456, ..., 1.01649811, 0.95302842,
        0.97373049],
       [1.00197828, 1.00531563, 1.01086955, ..., 1.0225131 , 1.03761348,
        1.01045947]])

In [13]:
def getTotalReward(ys, actions):
    reward = 0
    n = len(actions)

    assert len(ys) == n
    assert len(actions[0]) == numStocksInSubset
    assert len(ys) == numStocksInSubset

    originalWeights = actions
    inflatedWeights = []
    updatedWeights = [np.zeros(len(actions[0]))]
    for index, currWeights in enumerate(actions):
        inflatedWeights.append(currWeights * ys[index])
        updatedWeights.append(inflatedWeights[-1] / (currWeights @ ys[index]))

    for index in range(n):
        tranCost = tranCostRate * abs(originalWeights[index] - updatedWeights[index]).sum()
        reward += np.log(inflatedWeights[index] * (1 - tranCost))
    
    reward /= n

    return reward

In [14]:
def runModel(modelInstance, encInput, decInput, prevAction):
    return modelInstance.forward(encInput, decInput, prevAction)

In [15]:
modelInstance = RATransformer(1, k, 4, 12, 2, l)
optimizer = optim.Adam(modelInstance.parameters(),lr=1e-2)
for _ in range(numEpisodes):
    randomSubsets = [random.sample(range(numTickers), numStocksInSubset) for _ in range(numBatches)]
    ys = [stockInflations[k:] for stockInflations in inflations[randomSubsets]] # shape: (numDates-1: T-1, numStocks: m)
    actions = [np.zeros(4)]
    for i in range(numDates - k):
        print(i)
        encInput = [[priceSeries[i:i+k] for priceSeries in entryArrays[randomSubset]] for randomSubset in randomSubsets] # shape: (numBatches, numStocksInSubset, priceSeriesLength: k, numFeatures)
        encInput = torch.Tensor(encInput)
        decInput = [[priceSeries[i+k-l:i+k] for priceSeries in entryArrays[randomSubset]] for randomSubset in randomSubsets] # shape: (numBatches, numStocksInSubset, localContextLength: l, numFeatures)
        decInput = torch.Tensor(decInput)
        actions.append(runModel(modelInstance, encInput, decInput, actions[-1]))
    totalReward = getTotalReward(ys, actions)

    optimizer.zero_grad()
    totalReward.backward()
    optimizer.step()

0


AssertionError: Torch not compiled with CUDA enabled